In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
from scipy import integrate
from scipy.stats import linregress
from matplotlib import cm

In [2]:
#Constants
a=6371220 #radius of earth in m
L=2.5E6 #Latent heat in atmosphere
L_ice=3.34e5 #Latent heat of fusion
g=9.81 #Acceleration from gravity
conv_pw=1e15 #Conversion from watts to PW
cp=1007  

In [3]:
#Make some standardized coordinates

lats = np.arange(-90, 91, 1)
geometry = 2 * np.pi * np.cos(np.deg2rad(np.asarray(lats))) * a / g
levels_pa = np.array([100, 500, 1000, 2000, 3000, 5000, 7000, 10000, 15000, 20000, 25000, 30000, 40000, 50000, 60000,
             70000, 85000, 92500, 100000])
levels_hPa = levels_pa / 100

#### CURRENT BAD FILES ARE ALL GISS AND FGOALS-g3

### Open CMIP data

In [4]:
def open_aht(file, historical=False):

    final_times = pd.date_range('1980-01-01', '2014-12-31', freq='1M')
    
    final_lat = lats
    
    if historical == True:
        try:
            possible_ssp_file = file.replace('historical','ssp370')
            aht_ds = xr.open_mfdataset([file, possible_ssp_file])
        except FileNotFoundError:
            aht_ds = xr.open_dataset(file)

    else:
        aht_ds = xr.open_dataset(file)
    
    aht_ds = aht_ds.sel(time=slice('1980-01-01','2014-12-31'))

    aht_ds = aht_ds.interp(latitude=final_lat)
    
    try:
        aht_ds['time'] = final_times
    except ValueError:
        aht_ds = aht_ds.sel(time=slice('1980-01-01','2014-12-31'))
        aht_ds['time'] =  pd.date_range('1980-01-01', '2014-12-31', freq='1M')
    except:
        pass
    
    return aht_ds

In [24]:
#AMIP

aht_files_amip = glob.glob('../CMIP6/*_amip_aht_ds.nc')
avoid_files_amip = ['FGOALS-g3']#, 'GISS']

all_ahts_amip = []
model_list_amip = []

for file in aht_files_amip:
    if any(x in file for x in avoid_files_amip):
        print(f'Skipping {file}')
        
    else:
        all_ahts_amip.append(open_aht(file))
        model_list_amip.append(file.split('_')[0].split('/')[-1])
        
amip_ds = xr.concat(all_ahts_amip, pd.Index(model_list_amip, name='model'))

Skipping ../CMIP6/FGOALS-g3_amip_aht_ds.nc


In [25]:
#Historical

aht_files_hist = glob.glob('../CMIP6/*_historical_aht_ds.nc')
avoid_files_hist = ['FGOALS-g3']#, 'GISS']

all_ahts_hist = []
model_list_hist = []

for file in aht_files_hist:
    if any(x in file for x in avoid_files_hist):
        print(f'Skipping {file}')
        
    else:
        all_ahts_hist.append(open_aht(file, historical=True))
        model_list_hist.append(file.split('_')[0].split('/')[-1])
        
hist_ds = xr.concat(all_ahts_hist, pd.Index(model_list_hist, name='model'))

hist_ds = hist_ds.compute()

Skipping ../CMIP6/FGOALS-g3_historical_aht_ds.nc


In [26]:
### TEMP FIX FOR GISS ###
hist_ds['mmc_aht'][:,:,-3:] = 0
hist_ds['mmc_aht'][:,:,:3] = 0

hist_ds['mmc_aht_dry'][:,:,-3:] = 0
hist_ds['mmc_aht_dry'][:,:,:3] = 0

hist_ds['mmc_aht_dry_pot'][:,:,-3:] = 0
hist_ds['mmc_aht_dry_pot'][:,:,:3] = 0

hist_ds['mmc_aht_pot'][:,:,-3:] = 0
hist_ds['mmc_aht_pot'][:,:,:3] = 0

hist_ds['mmc_aht_moist'][:,:,-3:] = 0
hist_ds['mmc_aht_moist'][:,:,:3] = 0

hist_ds['te_aht'][:,:,-3:] = 0
hist_ds['te_aht'][:,:,:3] = 0

hist_ds['te_aht_dry_pot'][:,:,-3:] = 0
hist_ds['te_aht_dry_pot'][:,:,:3] = 0

hist_ds['te_aht_moist'][:,:,-3:] = 0
hist_ds['te_aht_moist'][:,:,:3] = 0

### TEMP FIX FOR GISS ###
amip_ds['mmc_aht'][:,:,-3:] = 0
amip_ds['mmc_aht'][:,:,:3] = 0

amip_ds['mmc_aht_dry'][:,:,-3:] = 0
amip_ds['mmc_aht_dry'][:,:,:3] = 0

amip_ds['mmc_aht_dry_pot'][:,:,-3:] = 0
amip_ds['mmc_aht_dry_pot'][:,:,:3] = 0

amip_ds['mmc_aht_pot'][:,:,-3:] = 0
amip_ds['mmc_aht_pot'][:,:,:3] = 0

amip_ds['mmc_aht_moist'][:,:,-3:] = 0
amip_ds['mmc_aht_moist'][:,:,:3] = 0

amip_ds['te_aht'][:,:,-3:] = 0
amip_ds['te_aht'][:,:,:3] = 0

amip_ds['te_aht_dry_pot'][:,:,-3:] = 0
amip_ds['te_aht_dry_pot'][:,:,:3] = 0

amip_ds['te_aht_moist'][:,:,-3:] = 0
amip_ds['te_aht_moist'][:,:,:3] = 0

#Temp fix as eddy AHT is weird
amip_ds['eddy_aht'] = amip_ds.se_aht + amip_ds.te_aht
amip_ds['eddy_aht_moist'] = amip_ds.se_aht_moist + amip_ds.te_aht_moist
amip_ds['eddy_aht_dry_pot'] = amip_ds.se_aht_dry_pot + amip_ds.te_aht_dry_pot

hist_ds['eddy_aht'] = hist_ds.se_aht + hist_ds.te_aht
hist_ds['eddy_aht_moist'] = hist_ds.se_aht_moist + hist_ds.te_aht_moist
hist_ds['eddy_aht_dry_pot'] = hist_ds.se_aht_dry_pot + hist_ds.te_aht_dry_pot

In [27]:
#amip_ds.to_netcdf('amip_aht.nc')
#hist_ds.to_netcdf('hist_aht.nc')

### Load in CESM2 LE runs

In [14]:
#All CESM runs are offset by one month - need to fix this

cesm2_time = pd.date_range("1950-01-01","2014-12-31",freq='1M')
cesm2_le_ds_full = xr.open_dataset('../CESM_aht/CESM2_LENS2_aht_ds.nc')
cesm2_le_ds_full['time'] = cesm2_time
cesm2_le_ds = cesm2_le_ds_full.sel(time=slice('1980-01-01','2014-12-31'))

cesm2_amip_time = pd.date_range("1949-12-31","2014-11-30",freq='1M')

cesm2_amip_le_ds = xr.open_dataset('../CESM_aht/CESM2_GOGA_AMIP_aht_ds.nc')
cesm2_amip_le_ds['time'] = cesm2_amip_time
cesm2_amip_le_ds = cesm2_amip_le_ds.sel(time=slice('1980-01-01','2014-12-31'))

cesm2_toga_amip_le_ds = xr.open_dataset('../CESM_aht/CESM2_TOGA_AMIP_aht_ds.nc')
cesm2_toga_amip_le_ds['time'] = cesm2_amip_time
cesm2_toga_amip_le_ds = cesm2_toga_amip_le_ds.sel(time=slice('1980-01-01','2014-12-31'))

cesm2_le_ds.to_netcdf('CESM2_LENS2_aht.nc')
cesm2_amip_le_ds.to_netcdf('CESM2_GOGA_aht.nc')
cesm2_toga_amip_le_ds.to_netcdf('CESM2_TOGA_aht.nc')

### Load in reanalysis

In [17]:
reanalysis_time_slice = slice('1980-01-01','2014-12-31')

era5_ds = xr.open_dataset('../era5_data/era5_aht_ds_1979_2022.nc').sel(time=reanalysis_time_slice)
#era_interim_ds = xr.open_dataset('../era_interim/era_interim_aht.nc').sel(time=reanalysis_time_slice)
merra_ds = xr.open_dataset('../merra_data/merra_aht_1980_2022.nc').sel(time=reanalysis_time_slice)
jra_ds = xr.open_dataset('../JRA_aht/jra_aht_1959_2022.nc').sel(time=reanalysis_time_slice)


#Times mean the same thing, this makes them actually identical
merra_ds['time'] = era5_ds.time
jra_ds['time'] = era5_ds.time

#For now merra strm_fnct has bad coordinate name
merra_strm = merra_ds.strm_fnct.values
merra_ds = merra_ds.drop('strm_fnct')
merra_ds = merra_ds.assign(dict(strm_fnct=(['time','level', 'latitude'], merra_strm)))

#Now interp everything to common coordinates
era5_ds = era5_ds.interp(latitude=lats, level=levels_hPa)
#era_interim_ds = era_interim_ds.interp(latitude=lats, level=levels_hPa)
merra_ds = merra_ds.interp(latitude=lats, level=levels_hPa)
jra_ds = jra_ds.interp(latitude=lats, level=levels_hPa)

ds_vars = ['total_aht', 'total_aht_dry', 'total_aht_dry_pot', 'total_aht_moist',
           'te_aht', 'te_aht_dry', 'te_aht_dry_pot', 'te_aht_moist',
           'se_aht', 'se_aht_dry', 'se_aht_dry_pot', 'se_aht_moist',
           'mmc_aht', 'mmc_aht_dry', 'mmc_aht_dry_pot', 'mmc_aht_moist',
           'eddy_aht', 'eddy_aht_dry', 'eddy_aht_dry_pot', 'eddy_aht_moist',
           'strm_fnct', 'mse_surf']

#reanalysis_ds = xr.concat([era5_ds[ds_vars], era_interim_ds[ds_vars], merra_ds[ds_vars], jra_ds[ds_vars]],
                          #pd.Index(['ERA5', 'ERA-Interim', 'MERRA', 'JRA'], name='model'))
reanalysis_ds = xr.concat([era5_ds[ds_vars],  merra_ds[ds_vars], jra_ds[ds_vars]],
                          pd.Index(['ERA5',  'MERRA', 'JRA'], name='model'))



In [18]:
reanalysis_ds

<xarray.Dataset>
Dimensions:            (model: 3, time: 420, latitude: 181, level: 19)
Coordinates:
  * time               (time) datetime64[ns] 1980-01-31 ... 2014-12-31
  * latitude           (latitude) int64 -90 -89 -88 -87 -86 ... 86 87 88 89 90
  * level              (level) float64 1.0 5.0 10.0 20.0 ... 850.0 925.0 1e+03
  * model              (model) object 'ERA5' 'MERRA' 'JRA'
Data variables: (12/22)
    total_aht          (model, time, latitude) float64 -1.338e-05 ... -0.0009766
    total_aht_dry      (model, time, latitude) float64 -1.448e-05 ... 0.0
    total_aht_dry_pot  (model, time, latitude) float64 -1.305e-05 ... 0.0009766
    total_aht_moist    (model, time, latitude) float64 -3.305e-07 ... 0.0
    te_aht             (model, time, latitude) float64 1.796e-06 ... 0.0
    te_aht_dry         (model, time, latitude) float64 1.379e-06 ... 0.0
    ...                 ...
    eddy_aht           (model, time, latitude) float64 1.796e-06 ... 0.0
    eddy_aht_dry       (model, time, latitude) float64 1.379e-06 ... 0.0
    eddy_aht_dry_pot   (model, time, latitude) float64 1.69e-06 ... 0.0009766
    eddy_aht_moist     (model, time, latitude) float64 1.06e-07 ... 6.104e-05
    strm_fnct          (model, time, level, latitude) float64 -0.00695 ... 0.0
    mse_surf           (model, time, latitude) float64 2.663e+05 ... 2.523e+05

In [19]:
#reanalysis_ds.to_netcdf('../AHT_trends_work/reanalysis_aht.nc')